In [21]:
## imports
import sys
import os
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import gridspec
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import eofs
from eofs.xarray import Eof
import cftime
import nc_time_axis

import mtspec

# from pylab import *

import datetime
import time
import glob, os
import math
import matplotlib.colors as colors
import xesmf as xe
from defs import regularAtlmask

In [1]:
datafolder = ''

In [29]:
experiment = 'Eoi400'
model = 'IPSL-CM5A'
filepath = datafolder+'Raw_data/'+model+'/'

In [30]:
if experiment == 'E280':
    dsT = xr.open_dataset(filepath+experiment+'/IPSLCM5A_piControl_oceTemp_oceSal_V.nc')
    dsV = xr.open_dataset(filepath+experiment+'/IPSLCM5A_piControl_oceTemp_oceSal_V.nc')
    dsS = xr.open_dataset(filepath+experiment+'/IPSLCM5A_piControl_oceTemp_oceSal_V.nc')
else:
    dsV = xr.open_dataset(filepath+experiment+'/IPSLCM5A_Eoi400_oceTemp_oceSal_V.nc', decode_times=False)
    dsT = xr.open_dataset(filepath+experiment+'/IPSLCM5A_Eoi400_oceTemp_oceSal_V.nc', decode_times=False)
    dsS = xr.open_dataset(filepath+experiment+'/IPSLCM5A_Eoi400_oceTemp_oceSal_V.nc', decode_times=False)
dsgrid = xr.open_dataset(datafolder+'Raw_data/CCSM4/'+experiment+'/'+experiment+'.SST.mean_month.nc', decode_times=False)

In [31]:
dsV = dsV.drop(['deptht','time_counter_bnds'])
dsV = dsV.drop_dims(['deptht'])
dsV = dsV.rename({'nav_lon': 'longitude', 'nav_lat': 'latitude', 'depthv':'level'})

dsT = dsT.drop(['depthv','time_counter_bnds'])
dsT = dsT.drop_dims(['depthv'])
dsT = dsT.rename({'nav_lon': 'longitude', 'nav_lat': 'latitude', 'deptht':'level'})

dsS = dsS.drop(['depthv','time_counter_bnds'])
dsS = dsS.drop_dims(['depthv'])
dsS = dsS.rename({'nav_lon': 'longitude', 'nav_lat': 'latitude', 'deptht':'level'})


In [33]:
regridder = xe.Regridder(dsS, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsS.vosaline.mean(dim='time_counter'))

ds = xr.Dataset(data_vars = dict(so=(("z","lat","lon"), var)), coords = dict(z = var.level.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/so_1x1_100yr.nc')

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


In [34]:
regridder = xe.Regridder(dsV, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsV.vomecrty.mean(dim='time_counter')*100)

ds = xr.Dataset(data_vars = dict(vo=(("z","lat","lon"), var)), coords = dict(z = var.level.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/vo_1x1_100yr.nc')

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


In [35]:
regridder = xe.Regridder(dsT, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsT.votemper.mean(dim='time_counter'))

ds = xr.Dataset(data_vars = dict(thetao=(("z","lat","lon"), var)), coords = dict(z = var.level.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/thetao_1x1_100yr.nc')

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
